# פרויקט אמצע סמסטר - ניתוח נתונים
### מתן אלמליח 205625221 | דניאל פרץ

---

**הבעיה העסקית:** בחינת הקשר בין מאפייני הסרט (ז'אנר ודירוג איכות) לבין הצלחתו הכלכלית (Gross Revenue).

**מטרת האנליזה:** הדאטה-סט מכיל את 1,000 הסרטים המובילים בדירוג. המטרה שלנו היא לאפיין את האוכלוסייה הזו ולבדוק האם סרטים בעלי דירוג איכות גבוה יותר, או מז'אנרים מסוימים, נוטים להרוויח יותר בקופות.

**מהלך העבודה:**
* **ניקוי נתונים:** המרת נתונים טקסטואליים (כסף וזמן) למספרים וטיפול בערכים חסרים וחריגים.
* **ניתוח חד-משתני (Uni-variate):** מיפוי התפלגות הציונים והז'אנרים השכיחים.
* **ניתוח רב-משתני (בונוס):** הצלבת הנתונים לזיהוי קשר בין איכות, ז'אנר והכנסות.

In [ ]:
# ייבוא ספריותimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as sns

---
## סעיף 1: טעינת הנתונים
העלאת קובץ הנתונים לפייתון

In [ ]:
# טעינת הנתונים מקובץ CSVdf = pd.read_csv('imdb_top_1000.csv')

---
## סעיף 2: הצגת רשומות
סקירה ראשונית של הנתונים

In [ ]:
# הצגת 5 השורות הראשונותdf.head()

In [ ]:
# הצגת מידע על סוגי הנתונים וערכים חסריםdf.info()

---
## סעיף 3: בדיקת וטיפול בכפילויות
זיהוי והסרת שורות כפולות

In [ ]:
# בדיקת מספר השורות הכפולותdup_count = df.duplicated().sum()print(f"מספר השורות הכפולות בדאטה הוא: {dup_count}")

In [ ]:
# הסרת שורות כפולות# inplace=True שומר את השינוי ישירות בדאטהפרייםdf.drop_duplicates(inplace=True)

---
## סעיף 4: שינוי ותיקון שמות עמודות
עדכון שמות עמודות לשמות ברורים יותר

In [ ]:
# שינוי שם העמודה Series_Title לשם ברור יותרdf.rename(columns={'Series_Title': 'Movie_Name'}, inplace=True)df.head(2)

---
## סעיף 5: טיפול בערכים לא תקינים
המרת נתונים טקסטואליים למספרים

In [ ]:
# המרת עמודת Runtime: הסרת המילה 'min' והמרה למספר שלםdf['Runtime'] = df['Runtime'].astype(str).str.replace(' min', '').astype(int)# המרת עמודת Gross: הסרת פסיקים (לדוגמה: '1,000' הופך ל-'1000')df['Gross'] = df['Gross'].str.replace(',', '')# המרה למספר - errors='coerce' הופך ערכים לא תקינים ל-NaN במקום לקרוסdf['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')df.info()

---
## סעיף 6: טיפול בערכים חסרים
מילוי ערכים חסרים בשיטות מתאימות

In [ ]:
# חישוב החציון של עמודת ההכנסותGross_median = df['Gross'].median()# מילוי ערכים חסרים בחציוןdf['Gross'] = df['Gross'].fillna(Gross_median)# בדיקה שאין יותר ערכים חסריםprint(f"מספר הערכים החסרים בעמודת Gross: {df['Gross'].isnull().sum()}")

In [ ]:
# טיפול בערכים חסרים בעמודות הנותרות# Meta_score: מילוי בחציוןMeta_median = df['Meta_score'].median()df['Meta_score'] = df['Meta_score'].fillna(Meta_median)# Certificate: מילוי בערך 'Unknown' (לא ידוע)df['Certificate'] = df['Certificate'].fillna('Unknown')# בדיקת מצב הנתונים הסופיdf.info()

---
## סעיף 7: סיכום סטטיסטי של הנתונים
סטטיסטיקה תיאורית למשתנים הנומריים

In [ ]:
# הצגת סטטיסטיקה תיאורית# כולל: ממוצע, סטיית תקן, מינימום, מקסימום, חציון ורבעוניםdf.describe()

---
## סעיף 8: זיהוי וטיפול בערכי קיצון (Outliers)
איתור ערכים חריגים וטיפול בהם

In [ ]:
# זיהוי ערכי קיצון בשיטת IQR (טווח בין-רבעוני)# נבדוק את העמודות הנומריות: Gross, Runtime, Meta_score, No_of_Votesnumerical_cols = ['Gross', 'Runtime', 'Meta_score', 'No_of_Votes']def detect_outliers_iqr(df, column):    """פונקציה לזיהוי ערכי קיצון בשיטת IQR"""    Q1 = df[column].quantile(0.25)  # רבעון ראשון    Q3 = df[column].quantile(0.75)  # רבעון שלישי    IQR = Q3 - Q1  # טווח בין-רבעוני    lower_bound = Q1 - 1.5 * IQR  # גבול תחתון    upper_bound = Q3 + 1.5 * IQR  # גבול עליון    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]    return len(outliers), lower_bound, upper_boundprint('=== זיהוי ערכי קיצון בשיטת IQR ===')print()for col in numerical_cols:    count, lower, upper = detect_outliers_iqr(df, col)    print(f'{col}:')    print(f'  גבול תחתון: {lower:,.2f}')    print(f'  גבול עליון: {upper:,.2f}')    print(f'  מספר ערכי קיצון: {count}')    print()

In [ ]:
# ויזואליזציה של ערכי קיצון באמצעות Boxplotsfig, axes = plt.subplots(2, 2, figsize=(14, 10))# גרף קופסה להכנסותsns.boxplot(data=df, y='Gross', ax=axes[0, 0], color='steelblue')axes[0, 0].set_title('התפלגות הכנסות (Gross Revenue)')axes[0, 0].set_ylabel('הכנסות ($)')# גרף קופסה לאורך הסרטsns.boxplot(data=df, y='Runtime', ax=axes[0, 1], color='coral')axes[0, 1].set_title('התפלגות אורך הסרט (Runtime)')axes[0, 1].set_ylabel('דקות')# גרף קופסה לציון Metasns.boxplot(data=df, y='Meta_score', ax=axes[1, 0], color='forestgreen')axes[1, 0].set_title('התפלגות ציון Meta')axes[1, 0].set_ylabel('ציון')# גרף קופסה למספר הצבעותsns.boxplot(data=df, y='No_of_Votes', ax=axes[1, 1], color='orchid')axes[1, 1].set_title('התפלגות מספר הצבעות')axes[1, 1].set_ylabel('מספר הצבעות')plt.tight_layout()plt.suptitle('זיהוי ערכי קיצון באמצעות Boxplots', y=1.02, fontsize=14)plt.show()

In [ ]:
# טיפול בערכי קיצון בשיטת Capping# במקום למחוק ערכים קיצוניים, נגביל אותם לגבולות ה-IQR# זה שומר על כמות הנתונים תוך הפחתת השפעת הערכים הקיצונייםdef cap_outliers(df, column):    """פונקציה להגבלת ערכי קיצון לגבולות IQR"""    Q1 = df[column].quantile(0.25)    Q3 = df[column].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR        # הגבלת הערכים    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)    return df# נטפל בעמודות Gross ו-No_of_Votes (שיש בהן הכי הרבה ערכי קיצון)# Runtime ו-Meta_score נשאיר כמו שהם כי הערכים הקיצוניים שלהם משמעותייםprint('=== טיפול בערכי קיצון ===')print('מבצעים Capping לעמודות Gross ו-No_of_Votes')print()# שמירת הנתונים לפני הטיפול להשוואהprint('לפני הטיפול:')print(f"Gross - מקסימום: {df['Gross'].max():,.0f}, מינימום: {df['Gross'].min():,.0f}")print(f"No_of_Votes - מקסימום: {df['No_of_Votes'].max():,}, מינימום: {df['No_of_Votes'].min():,}")# הפעלת הטיפולdf = cap_outliers(df, 'Gross')df = cap_outliers(df, 'No_of_Votes')print()print('אחרי הטיפול:')print(f"Gross - מקסימום: {df['Gross'].max():,.0f}, מינימום: {df['Gross'].min():,.0f}")print(f"No_of_Votes - מקסימום: {df['No_of_Votes'].max():,}, מינימום: {df['No_of_Votes'].min():,}")

In [ ]:
# גרפי קופסה לאחר הטיפול בערכי קיצוןfig, axes = plt.subplots(1, 2, figsize=(12, 5))sns.boxplot(data=df, y='Gross', ax=axes[0], color='steelblue')axes[0].set_title('הכנסות (לאחר Capping)')axes[0].set_ylabel('הכנסות ($)')sns.boxplot(data=df, y='No_of_Votes', ax=axes[1], color='orchid')axes[1].set_title('מספר הצבעות (לאחר Capping)')axes[1].set_ylabel('מספר הצבעות')plt.tight_layout()plt.suptitle('Boxplots לאחר טיפול בערכי קיצון', y=1.02, fontsize=14)plt.show()

---
## סעיף 9: ניתוח חד-משתני (Univariate Analysis)
ניתוח והצגה של כל משתנה בנפרד

In [ ]:
# היסטוגרמות למשתנים הנומריים# מציגות את התפלגות הערכים בכל משתנהfig, axes = plt.subplots(2, 2, figsize=(14, 10))# התפלגות הכנסותsns.histplot(df['Gross'], bins=20, kde=True, ax=axes[0, 0], color='steelblue')axes[0, 0].set_title('התפלגות הכנסות')axes[0, 0].set_xlabel('הכנסות ($)')axes[0, 0].set_ylabel('מספר סרטים')# התפלגות אורך הסרטsns.histplot(df['Runtime'], bins=20, kde=True, ax=axes[0, 1], color='coral')axes[0, 1].set_title('התפלגות אורך הסרט')axes[0, 1].set_xlabel('דקות')axes[0, 1].set_ylabel('מספר סרטים')# התפלגות ציון Metasns.histplot(df['Meta_score'], bins=20, kde=True, ax=axes[1, 0], color='forestgreen')axes[1, 0].set_title('התפלגות ציון Meta')axes[1, 0].set_xlabel('ציון')axes[1, 0].set_ylabel('מספר סרטים')# התפלגות מספר הצבעותsns.histplot(df['No_of_Votes'], bins=20, kde=True, ax=axes[1, 1], color='orchid')axes[1, 1].set_title('התפלגות מספר הצבעות')axes[1, 1].set_xlabel('מספר הצבעות')axes[1, 1].set_ylabel('מספר סרטים')plt.tight_layout()plt.suptitle('התפלגות המשתנים הנומריים', y=1.02, fontsize=14)plt.show()

In [ ]:
# התפלגות דירוג IMDBplt.figure(figsize=(10, 5))sns.histplot(df['IMDB_Rating'], bins=15, kde=True, color='gold')plt.title('התפלגות דירוגי IMDB')plt.xlabel('דירוג')plt.ylabel('מספר סרטים')plt.show()

In [ ]:
# התפלגות סרטים לפי עשור# המרת שנת היציאה למספרdf['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')# יצירת עמודת עשורdf['Decade'] = (df['Released_Year'] // 10) * 10plt.figure(figsize=(12, 6))decade_counts = df['Decade'].value_counts().sort_index()sns.barplot(x=decade_counts.index.astype(int), y=decade_counts.values, palette='viridis')plt.title('התפלגות סרטים לפי עשור')plt.xlabel('עשור')plt.ylabel('מספר סרטים')plt.xticks(rotation=45)plt.show()

In [ ]:
# התפלגות ז'אנרים# פיצול מחרוזת הז'אנרים ולקיחת הז'אנר הראשי בלבדdf['main_Genre'] = df['Genre'].astype(str).str.split(',').str[0]plt.figure(figsize=(12, 6))# הצגת 10 הז'אנרים הנפוצים ביותרsns.countplot(    data=df,    x='main_Genre',    order=df['main_Genre'].value_counts().index[:10],    palette='Set2')plt.title("10 הז'אנרים הנפוצים ביותר")plt.xlabel("ז'אנר")plt.ylabel('מספר סרטים')plt.xticks(rotation=45)plt.show()

---
## בונוס: ניתוח רב-משתני (Multivariate Analysis)
בחינת הקשר בין מספר משתנים יחד

In [ ]:
# גרף פיזור המציג את הקשר בין דירוג, הכנסות וז'אנר# יצירת עמודה של הכנסות במיליונים לקריאות טובה יותרdf['Gross_millions'] = df['Gross'] / 1000000plt.figure(figsize=(12, 8))# גרף פיזור עם 3 משתנים:# ציר X = דירוג IMDB# ציר Y = הכנסות# צבע = ז'אנרsns.scatterplot(    data=df,    x='IMDB_Rating',    y='Gross_millions',    hue='main_Genre',    s=100,  # גודל הנקודות    alpha=0.7  # שקיפות (עוזר כשנקודות חופפות))plt.title("הקשר בין דירוג IMDB, הכנסות וז'אנר")plt.xlabel('דירוג IMDB')plt.ylabel('הכנסות (מיליוני $)')plt.legend(title="ז'אנר", bbox_to_anchor=(1.05, 1), loc='upper left')plt.tight_layout()plt.show()